In [1]:
### Import packages ###
import ast
import itertools
import numpy as np
import pandas as pd

# Input Data Set 

In [2]:
### Regression ###
# Data =["Auto"]
# Data =["BostonHousing"]
# Data =["CPS"]
# Data =["Simulate"]
# Data =["Yacht"]

### Classification ###
Data =["BostonHousingBinned"]


# Input directory

In [3]:
# Directory
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Rashomon Parameters

In [4]:
import itertools
import pandas as pd

# Rashomon QBC parameters
Seed = range(0, 100)
Data = ["BostonHousingBinned"]  # Example Data
TestProportion = [0.2]
CandidateProportion = [0.8]
SelectorType = ["TreeEnsembleQBCFunction"]
ModelType = ["TreeFarmsFunction"]
DataArgs = [{"N": 1000, "K": 5}]
SelectorArgs = [
    {"TopCModels": 1, "UniqueErrorsInput": 1},
    {"TopCModels": 1, "UniqueErrorsInput": 0}
]
ModelArgs = [
    {"n_estimators": 100,
     "Seed": 1,
     "TopCModels": 1, 
     "config": {"regularization": 0.01, "rashomon_bound_multiplier": 0.1}, "Type": "Classification"}
]


# Generate filtered combinations
filtered_combinations = [
    (seed, data, test_prop, cand_prop, selector_type, model_type, data_arg, sel_arg, mod_arg)
    for seed, data, test_prop, cand_prop, selector_type, model_type, data_arg, sel_arg, mod_arg in itertools.product(
        Seed, Data, TestProportion, CandidateProportion, SelectorType, ModelType, DataArgs, SelectorArgs, ModelArgs
    )
    if (model_type == "TreeFarmsFunction" and sel_arg["TopCModels"] == mod_arg["TopCModels"]) or
       (model_type == "RandomForestClassificationFunction" and 
        sel_arg["TopCModels"] == 10 and 
        mod_arg["TopCModels"] == 10 and 
        sel_arg["UniqueErrorsInput"] == 1) 
]

# Create DataFrame
df = pd.DataFrame(filtered_combinations, columns=[
    "Seed", "Data", "TestProportion", "CandidateProportion", "SelectorType", 
    "ModelType", "DataArgs", "SelectorArgs", "ModelArgs"
])


# Create JobName and Output

In [5]:
# Replacement dictionary for formatting
ReplacementCharacters = {":": "",  " ": "", "{": "", "}": "", "\"": "", ",": "_", "[": "", "]": ""}

# Generate JobName
df["JobName"] = (
    "Seed" + df["Seed"].astype(str) +
    "_Data" + df["Data"].astype(str) +
    "_TP" + df["TestProportion"].astype(str) +
    "_CP" + df["CandidateProportion"].astype(str) +
    "_ST" + df["SelectorType"].astype(str) +
    "_MT" + df["ModelType"].astype(str) +
    "_UEI" + df["SelectorArgs"].apply(lambda x: str(x["UniqueErrorsInput"])) +
    "_RashomonNum" + df["SelectorArgs"].apply(lambda x: str(x["TopCModels"]))
)

### Output Name ###
df["JobName"] = df["JobName"].str.replace("Function", "", regex=False)
df["Output"] =  df["Data"].astype(str) + "/" + df["ModelType"].astype(str) + "/Raw/" + df["JobName"] + ".pkl"
df["Output"] = df["Output"].str.replace("Function", "", regex=False)

In [6]:
### Change types ###
df["DataArgs"] = df["DataArgs"].astype(str)
df["SelectorArgs"] = df["SelectorArgs"].astype(str)
df["ModelArgs"] = df["ModelArgs"].astype(str)

df["DataArgs"] = "'"+ df["DataArgs"].apply(lambda x: str(x).replace("'", '"')) +"'"
df["SelectorArgs"] = "'"+ df["SelectorArgs"].apply(lambda x: str(x).replace("'", '"')) +"'"
df["ModelArgs"] = "'"+ df["ModelArgs"].apply(lambda x: str(x).replace("'", '"')) +"'"


In [7]:
### Save Parameter Vector ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data[0]) + ".csv"
df.to_csv(output_path, index=False)

In [8]:
len(set(df["JobName"]))

200

In [9]:
len(set(df["Output"]))

200

In [10]:
df.shape

(200, 11)

In [11]:
df

,Seed,Data,TestProportion,CandidateProportion,SelectorType,ModelType,DataArgs,SelectorArgs,ModelArgs,JobName,Output
0,0,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 1}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed0_DataBostonHousingBinned_TP0.2_CP0.8_STTr...,BostonHousingBinned/TreeFarms/Raw/Seed0_DataBo...
1,0,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 0}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed0_DataBostonHousingBinned_TP0.2_CP0.8_STTr...,BostonHousingBinned/TreeFarms/Raw/Seed0_DataBo...
2,1,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 1}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed1_DataBostonHousingBinned_TP0.2_CP0.8_STTr...,BostonHousingBinned/TreeFarms/Raw/Seed1_DataBo...
3,1,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 0}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed1_DataBostonHousingBinned_TP0.2_CP0.8_STTr...,BostonHousingBinned/TreeFarms/Raw/Seed1_DataBo...
4,2,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 1}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed2_DataBostonHousingBinned_TP0.2_CP0.8_STTr...,BostonHousingBinned/TreeFarms/Raw/Seed2_DataBo...
...,...,...,...,...,...,...,...,...,...,...,...
195,97,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 0}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed97_DataBostonHousingBinned_TP0.2_CP0.8_STT...,BostonHousingBinned/TreeFarms/Raw/Seed97_DataB...
196,98,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 1}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed98_DataBostonHousingBinned_TP0.2_CP0.8_STT...,BostonHousingBinned/TreeFarms/Raw/Seed98_DataB...
197,98,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 0}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed98_DataBostonHousingBinned_TP0.2_CP0.8_STT...,BostonHousingBinned/TreeFarms/Raw/Seed98_DataB...
198,99,BostonHousingBinned,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,"'{""N"": 1000, ""K"": 5}'","'{""TopCModels"": 1, ""UniqueErrorsInput"": 1}'","'{""n_estimators"": 100, ""Seed"": 1, ""TopCModels""...",Seed99_DataBostonHousingBinned_TP0.2_CP0.8_STT...,BostonHousingBinned/TreeFarms/Raw/Seed99_DataB...
